In [ ]:
import torch 
from tqdm import tqdm 
from torch.utils.data import Dataset, DataLoader 
from torch.optim import Adam
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 

import sys 
import gc 

In [ ]:

# from kaggle_datasets import KaggleDatasets
# GCS_DS_PATH = KaggleDatasets().get_gcs_path("allenailongformerbase4096")

In [ ]:
# !gsutil ls $GCS_DS_PATH

In [ ]:
print("done")

In [ ]:
train = pd.read_csv('../input/feedback-prize-2021/train.csv')
IDS = train.id.unique()
MAX_LEN = 1024
from transformers import LongformerTokenizerFast, LongformerForTokenClassification
# THE TOKENS AND ATTENTION ARRAYS
tokenizer = LongformerTokenizerFast.from_pretrained("../input/allenailongformerbase4096/longformer")
train_tokens = np.zeros((len(IDS),MAX_LEN), dtype='int32')
train_attention = np.zeros((len(IDS),MAX_LEN), dtype='int32')

# THE 14 CLASSES FOR NER
lead_b = np.zeros((len(IDS),MAX_LEN))
lead_i = np.zeros((len(IDS),MAX_LEN))

position_b = np.zeros((len(IDS),MAX_LEN))
position_i = np.zeros((len(IDS),MAX_LEN))

evidence_b = np.zeros((len(IDS),MAX_LEN))
evidence_i = np.zeros((len(IDS),MAX_LEN))

claim_b = np.zeros((len(IDS),MAX_LEN))
claim_i = np.zeros((len(IDS),MAX_LEN))

conclusion_b = np.zeros((len(IDS),MAX_LEN))
conclusion_i = np.zeros((len(IDS),MAX_LEN))

counterclaim_b = np.zeros((len(IDS),MAX_LEN))
counterclaim_i = np.zeros((len(IDS),MAX_LEN))

rebuttal_b = np.zeros((len(IDS),MAX_LEN))
rebuttal_i = np.zeros((len(IDS),MAX_LEN))

# HELPER VARIABLES
train_lens = []
targets_b = [lead_b, position_b, evidence_b, claim_b, conclusion_b, counterclaim_b, rebuttal_b]
targets_i = [lead_i, position_i, evidence_i, claim_i, conclusion_i, counterclaim_i, rebuttal_i]
target_map = {'Lead':0, 'Position':1, 'Evidence':2, 'Claim':3, 'Concluding Statement':4,
             'Counterclaim':5, 'Rebuttal':6}

In [ ]:
# GET TEST TEXT IDS
files = os.listdir('../input/feedback-prize-2021/test')
TEST_IDS = [f.replace('.txt','') for f in files if 'txt' in f]
print('There are',len(TEST_IDS),'test texts.')

In [ ]:
# CONVERT TEST TEXT TO TOKENS
test_tokens = np.zeros((len(TEST_IDS),MAX_LEN), dtype='int32')
test_attention = np.zeros((len(TEST_IDS),MAX_LEN), dtype='int32')

for id_num in range(len(TEST_IDS)):
        
    # READ TRAIN TEXT, TOKENIZE, AND SAVE IN TOKEN ARRAYS    
    n = TEST_IDS[id_num]
    name = f'../input/feedback-prize-2021/test/{n}.txt'
    txt = open(name, 'r').read()
    tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
    test_tokens[id_num,] = tokens['input_ids']
    test_attention[id_num,] = tokens['attention_mask']

In [ ]:
class Model(torch.nn.Module):
    def __init__(self,backbone):
        super(Model, self).__init__()
        self.backbone = backbone
        self.linear_1 = torch.nn.Linear(15,128)
        self.relu = torch.nn.ReLU()
        self.linear_2 = torch.nn.Linear(128, 15)
        self.softmax = torch.nn.Softmax()
    
    def forward(self, inp, att):
        outs = backbone(inp,att)
        outs = self.linear_1(outs.logits)
        outs = self.relu(outs)
        outs = self.linear_2(outs)
        outs = self.softmax(outs)
        return outs 

In [ ]:
from transformers import LongformerTokenizerFast, LongformerForTokenClassification
# THE TOKENS AND ATTENTION ARRAYS
backbone =LongformerForTokenClassification.from_pretrained('../input/allenailongformerbase4096/longformer',num_labels = 15 )

In [ ]:
test_tokens, test_attention

In [ ]:
model = Model(backbone = backbone)
model.load_state_dict(torch.load("../input/model-weights-longformer-pytorch/model_weights7.pth"))
model.eval()

In [ ]:
class Test_data(torch.utils.data.Dataset):
    def __init__(self, token, attention):
        self.token = token 
        self.attention = attention 
#         self.targets = targets 
    def __len__(self):
        return len(self.token)
    
    def __getitem__(self, idx):
        tok = self.token[idx]
        att = self.attention[idx]
#         tt = self.targets[idx]
#         tt = torch.tensor(tt, dtype = torch.float)
        tok = torch.tensor(tok)
        att = torch.tensor(att)
        return [tok,att]

In [ ]:
test_d = Test_data(token = test_tokens , attention = test_attention)
test_f = torch.utils.data.DataLoader(test_d, batch_size = 6, shuffle = False )

In [ ]:
total_outs = []
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
for idx , data in tqdm(enumerate(test_f)):
    #= lights will guide you HOME and ignite your BONES and I'll try to FIX YOU!! 
    ins = data[0].to(device)
    att = data[1].to(device)
    outs = model(inp = ins , att = att)
    total_outs.append(outs)
    gc.collect()

In [ ]:
total_outs = total_outs[0].cpu().detach().numpy()

In [ ]:




final_preds = []
final_scores = []
raw_preds = total_outs.copy()

pred_class = np.argmax(raw_preds, axis=2)
pred_scrs = np.max(raw_preds, axis=2)
for pred, pred_scr in zip(pred_class, pred_scrs):
        pred = pred.tolist()
        pred_scr = pred_scr.tolist()
        final_preds.append(pred)
        final_scores.append(pred_scr)

# for j in range(len(test_samples)):
#     tt = [id_target_map[p] for p in final_preds[j][1:]]
#     tt_score = final_scores[j][1:]
#     test_samples[j]["preds"] = tt
#     test_samples[j]["pred_scores"] = tt_score

final_preds, final_scores

def _prepare_test_data_helper(tokenizer, ids):
    test_samples = []
    for idx in ids:
        filename = os.path.join("../input/feedback-prize-2021/", "test", idx + ".txt")
        with open(filename, "r") as f:
            text = f.read()

        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=False,
            return_offsets_mapping=True,
        )
        input_ids = encoded_text["input_ids"]
        offset_mapping = encoded_text["offset_mapping"]

        sample = {
            "id": idx,
            "input_ids": input_ids,
            "text": text,
            "offset_mapping": offset_mapping,
        }

        test_samples.append(sample)
    return test_samples


def prepare_test_data(df, tokenizer):
    test_samples = []
    ids = df["id"].unique()
    ids_splits = np.array_split(ids, 4)

    results = [_prepare_test_data_helper(tokenizer, idx) for idx in ids_splits]
    
    for result in results:
        test_samples.extend(result)

    return test_samples

df = pd.read_csv(os.path.join("../input/feedback-prize-2021/", "sample_submission.csv"))
df_ids = df["id"].unique()

test_samples = prepare_test_data(df,tokenizer = tokenizer)

pd.DataFrame(test_samples)

output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}
for j in range(len(test_samples)):
    tt = [ids_to_labels[p] for p in final_preds[j][1:]]
    tt_score = final_scores[j][1:]
    test_samples[j]["preds"] = tt
    test_samples[j]["pred_scores"] = tt_score

pd.DataFrame(test_samples)

def jn(pst, start, end):
    return " ".join([str(x) for x in pst[start:end]])


def link_evidence(oof):
    thresh = 1
    idu = oof['id'].unique()
    idc = idu[1]
    eoof = oof[oof['class'] == "Evidence"]
    neoof = oof[oof['class'] != "Evidence"]
    for thresh2 in range(26,27, 1):
        retval = []
        for idv in idu:
            for c in  ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement',
                   'Counterclaim', 'Rebuttal']:
                q = eoof[(eoof['id'] == idv) & (eoof['class'] == c)]
                if len(q) == 0:
                    continue
                pst = []
                for i,r in q.iterrows():
                    pst = pst +[-1] + [int(x) for x in r['predictionstring'].split()]
                start = 1
                end = 1
                for i in range(2,len(pst)):
                    cur = pst[i]
                    end = i
                    #if pst[start] == 205:
                    #   print(cur, pst[start], cur - pst[start])
                    if (cur == -1 and c != 'Evidence') or ((cur == -1) and ((pst[i+1] > pst[end-1] + thresh) or (pst[i+1] - pst[start] > thresh2))):
                        retval.append((idv, c, jn(pst, start, end)))
                        start = i + 1
                v = (idv, c, jn(pst, start, end+1))
                #print(v)
                retval.append(v)
        roof = pd.DataFrame(retval, columns = ['id', 'class', 'predictionstring']) 
        roof = roof.merge(neoof, how='outer')
        return roof

proba_thresh = {
    "Lead": 0,
    "Position": 0,
    "Evidence": 0,
    "Claim": 0,
    "Concluding Statement": 0,
    "Counterclaim": 0,
    "Rebuttal": 0,
}

min_thresh = {
    "Lead": 9,
    "Position": 5,
    "Evidence": 14,
    "Claim": 3,
    "Concluding Statement": 11,
    "Counterclaim": 6,
    "Rebuttal": 4,
}

submission = []
for sample_idx, sample in enumerate(test_samples):
    preds = sample["preds"]
    offset_mapping = sample["offset_mapping"]
    sample_id = sample["id"]
    sample_text = sample["text"]
    sample_input_ids = sample["input_ids"]
    sample_pred_scores = sample["pred_scores"]
    sample_preds = []

    if len(preds) < len(offset_mapping):
        preds = preds + ["O"] * (len(offset_mapping) - len(preds))
        sample_pred_scores = sample_pred_scores + [0] * (len(offset_mapping) - len(sample_pred_scores))
    
    idx = 0
    phrase_preds = []
    while idx < len(offset_mapping):
        start, _ = offset_mapping[idx]
        if preds[idx] != "O":
            label = preds[idx][2:]
        else:
            label = "O"
        phrase_scores = []
        phrase_scores.append(sample_pred_scores[idx])
        idx += 1
        while idx < len(offset_mapping):
            if label == "O":
                matching_label = "O"
            else:
                matching_label = f"I-{label}"
            if preds[idx] == matching_label:
                _, end = offset_mapping[idx]
                phrase_scores.append(sample_pred_scores[idx])
                idx += 1
            else:
                break
        if "end" in locals():
            phrase = sample_text[start:end]
            phrase_preds.append((phrase, start, end, label, phrase_scores))

    temp_df = []
    for phrase_idx, (phrase, start, end, label, phrase_scores) in enumerate(phrase_preds):
        word_start = len(sample_text[:start].split())
        word_end = word_start + len(sample_text[start:end].split())
        word_end = min(word_end, len(sample_text.split()))
        ps = " ".join([str(x) for x in range(word_start, word_end)])
        if label != "O":
            if sum(phrase_scores) / len(phrase_scores) >= proba_thresh[label]:
                if len(ps.split()) >= min_thresh[label]:
                    temp_df.append((sample_id, label, ps))
    
    temp_df = pd.DataFrame(temp_df, columns=["id", "class", "predictionstring"])
    submission.append(temp_df)

submission

submission = pd.concat(submission).reset_index(drop=True)
submission = link_evidence(submission)
submission.to_csv("submission.csv", index=False)

submission